In [ ]:
# Import relevant packages
import numpy as np
import pandas as pd
from keras.models import Sequential
#from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.layers import Input, Dense
from keras.activations import relu, softmax
from keras import  Model
import keras
import tensorflow as tf


---
---
Some Utility Functions

---
---

In [ ]:
class PreProcessing:

  data_train = np.array([])
  data_test = np.array([])
  labels_train = np.array([])
  labels_test = np.array([])
  unique_train_label = np.array([])
  map_train_label_indices = dict()

  def __init__(self,data_src):

    self.data_src = data_src

  def normalize(self,x):
    min_val = np.min(x)
    max_val = np.max(x)
    x = (x - min_val) / (max_val - min_val)
    return x

  def encode_labels(self, labels):
    # integer encode
    label_encoder = LabelEncoder()
    y_enc = label_encoder.fit_transform(labels)
    y = np.asarray(y_enc)
    inverted = label_encoder.inverse_transform(y_enc)
    print("Number of labels: ", len(y), " Number of unique labels: ", len(np.unique(y)))
    return (y, inverted)

  def read_dataset(self):
    X = []
    y = []
    all_data = pd.read_csv(self.data_src)
    labels = all_data.values[-1].tolist()
    labels2 = labels[1:]
    all_data2  = all_data.drop(all_data.tail(1).index,inplace=False)
    all_data3 = all_data2.drop(columns='Unnamed: 0', inplace=False)
    X = np.transpose(np.asarray(all_data3))
    y = labels2

    print('Dataset loaded successfully.')
    return X,y

  def read_raw_dataset(self, src):
    print("Loading Dataset from...", src)
    all_data = pd.read_csv(src)
    print('Dataset loaded successfully.')
    return(all_data)

  def preprocessing(self,train_test_ratio):
    X, y = self.read_dataset()
    labels = list(set(y))
    label_dict = dict(zip(labels, range(len(labels))))
    Y = np.asarray([label_dict[label] for label in y])
    #X = [self.normalize(x) for x in X]

    shuffle_indices = np.random.permutation(np.arange(len(y)))
    x_shuffled = []
    y_shuffled = []
    for index in shuffle_indices:
        x_shuffled.append(X[index])
        y_shuffled.append(Y[index])

    size_of_dataset = len(x_shuffled)
    n_train = int(np.ceil(size_of_dataset * train_test_ratio))
    return np.asarray(x_shuffled[0:n_train]), np.asarray(x_shuffled[n_train + 1:size_of_dataset]), np.asarray(
        y_shuffled[0:n_train]), np.asarray(y_shuffled[
                                            n_train + 1:size_of_dataset])

  def get_triplets(self):
    label_l, label_r = np.random.choice(self.unique_train_label, 2, replace=False)
    a, p = np.random.choice(self.map_train_label_indices[label_l],2, replace=False)
    n = np.random.choice(self.map_train_label_indices[label_r])
    return a, p, n

  def get_triplets_batch(self,n):
    idxs_a, idxs_p, idxs_n = [], [], []
    for _ in range(n):
        a, p, n = self.get_triplets()
        idxs_a.append(a)
        idxs_p.append(p)
        idxs_n.append(n)
    return self.images_train[idxs_a,:], self.images_train[idxs_p, :], self.images_train[idxs_n, :]

  def merge_gene_data(self, data1, data2):
    # prepare data fro merging
    data1.sort_values(by = ['Unnamed: 0'], inplace=True)
    data2.sort_values(by = ['Unnamed: 0'], inplace=True)
    # get the extracted columns
    x1_genes = data1['Unnamed: 0']
    #print(x1_genes)
    x2_genes = data2['Unnamed: 0']
    #print(x2_genes)

    # delete the genes columns
    del data1['Unnamed: 0']
    del data2['Unnamed: 0']
    # now get the column names as cells
    data1_cells  = list(data1.columns)
    data2_cells  = list(data2.columns)
    # transpose and normalize
    data1_t = np.transpose(data1)
    data2_t = np.transpose(data2)
    data1_t.reset_index(inplace=True, drop = True)
    data2_t.reset_index(inplace=True, drop = True)

    # normalize the features
    data_array1 = np.asarray(data1_t).astype('float32')
    data_array2 = np.asarray(data2_t).astype('float32')

    data1_tt = np.transpose(data_array1)
    data_scaled1  = [self.normalize(feature) for feature in data1_tt]

    data2_tt = np.transpose(data_array2)
    data_scaled2  = [self.normalize(feature) for feature in data2_tt]

    X1 = np.transpose(data_scaled1)
    X2 = np.transpose(data_scaled2)
    merged_data = np.concatenate((X1, X2), axis = 0)

    (n1, g1) = X1.shape
    (n2, g2) = X2.shape
    (n, g) = merged_data.shape
    print("Number of cells of X1: ", n1, " Number of genes of X1: ", g1)
    print("Number of cells of X2: ", n2, " Number of genes of X2: ", g2)
    print("Number of cells of merged data: ", n, " Number of genes of merged data: ", g)

    return X1, X2, merged_data

  def get_mnist(self):
    np.random.seed(1234) # set seed for deterministic ordering
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_all = np.concatenate((x_train, x_test), axis = 0)
    Y = np.concatenate((y_train, y_test), axis = 0)
    X = x_all.reshape(-1,x_all.shape[1]*x_all.shape[2])

    p = np.random.permutation(X.shape[0])
    X = X[p].astype(np.float32)*0.02
    Y = Y[p]
    return X, Y



In [ ]:
class Metrics:
  def __init__(self, y_true = None, y_pred = None, df = None, title = None):

    super(Metrics, self).__init__()

  def nmi(self, y_true, y_pred):
    return(normalized_mutual_info_score(y_true, y_pred))

  def ari(self, y_true, y_pred):
    return(adjusted_rand_score(y_true, y_pred))

  def acc(self, y_true, y_pred):
    """
    # Arguments
        y: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`

    # Return
        accuracy, in [0,1]
    """
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    from scipy.optimize import linear_sum_assignment as linear_assignment
    #from sklearn.utils.linear_assignment_ import linear_assignment
    ind = linear_assignment(w.max() - w)
    ind = np.asarray(ind)
    ind = np.transpose(ind)
    return sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

  def plot3D(self, df, title):
    my_dpi=96
    plt.figure(figsize=(480/my_dpi, 480/my_dpi), dpi=my_dpi)
    pca = PCA(n_components=3)
    df = pd.DataFrame(df)
    pca.fit(df)
    # Store results of PCA in a data frame
    result=pd.DataFrame(pca.transform(df), columns=['PCA%i' % i for i in range(3)], index=df.index)

    # Plot initialisation
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(result['PCA0'], result['PCA1'], result['PCA2'], s=60)

    # label the axes
    ax.set_title(title)
    plt.show()

  def get_embedded_features(self, model, weights, X):
    model.load_weights(weights)
    features = model.extract_feature(X)
    pd.DataFrame(features).to_csv('results/embedded_features.csv')

  def visualize_clusters(self, model, weights, X, y):
    # extract features from IDEC model
    #dec = IDEC(dims=[X.shape[-1], 500, 500, 2000, 20])
    model.load_weights(weights)
    features = model.extract_feature(X)

    # using tsne to get data points in 2d space
    tsne = TSNE(n_iter=100000)
    embed_points = tsne.fit_transform(features)

    # plot embedded 2d points
    fig = plt.figure()
    colors = ['darkorange', 'g', 'm', 'gray', 'b', 'c', 'burlywood', 'purple', 'k', 'r']
    marker=['x']*9+['+']
    for i in range(10):
        plt.scatter(embed_points[y == i, 0], embed_points[y == i, 1], c=colors[i], marker=marker[i], label=str(i))
        plt.xticks(())
        plt.yticks(())
    fig.savefig('fig-tsne.pdf', dpi=600)

    plt.show()


---
---
Load Data

---
---

In [ ]:
prep = PreProcessing('')
orig1 = prep.read_raw_dataset("https://media.githubusercontent.com/media/Mcpaeis/Liger/main/neumf/pbmc_10X_var.csv")
orig2 = prep.read_raw_dataset("https://media.githubusercontent.com/media/Mcpaeis/Liger/main/neumf/pbmc_SqW_var.csv")
_, _, origd = prep.merge_gene_data(orig1, orig2)
liger = prep.read_raw_dataset("https://raw.githubusercontent.com/Mcpaeis/Liger/main/neumf/data/LIGER_EMBEDDED.csv")
neumf = prep.read_raw_dataset("https://raw.githubusercontent.com/Mcpaeis/Liger/main/neumf/data/NEUMF_EMBEDDED.csv")
neunn = prep.read_raw_dataset("https://raw.githubusercontent.com/Mcpaeis/Liger/main/neumf/embedded_features-04-17-2021.csv")
clusters = prep.read_raw_dataset("https://media.githubusercontent.com/media/Mcpaeis/Liger/main/neumf/clusters.csv")
small_test_data = prep.read_raw_dataset("https://raw.githubusercontent.com/Mcpaeis/Liger/main/neumf/sp_cell_ortho_counts_matrix_RPANDA.csv")
small_test_data_v1 = prep.read_raw_dataset("https://raw.githubusercontent.com/Mcpaeis/Liger/main/neumf/sp_cell_ortho_counts_matrix_RPANDA_Regrouped.csv")

Loading Dataset from... https://media.githubusercontent.com/media/Mcpaeis/Liger/main/neumf/pbmc_10X_var.csv
Dataset loaded successfully.
Loading Dataset from... https://media.githubusercontent.com/media/Mcpaeis/Liger/main/neumf/pbmc_SqW_var.csv
Dataset loaded successfully.
Number of cells of X1:  2638  Number of genes of X1:  2814
Number of cells of X2:  3694  Number of genes of X2:  2814
Number of cells of merged data:  6332  Number of genes of merged data:  2814
Loading Dataset from... https://raw.githubusercontent.com/Mcpaeis/Liger/main/neumf/data/LIGER_EMBEDDED.csv
Dataset loaded successfully.
Loading Dataset from... https://raw.githubusercontent.com/Mcpaeis/Liger/main/neumf/data/NEUMF_EMBEDDED.csv
Dataset loaded successfully.
Loading Dataset from... https://raw.githubusercontent.com/Mcpaeis/Liger/main/neumf/embedded_features-04-17-2021.csv
Dataset loaded successfully.
Loading Dataset from... https://media.githubusercontent.com/media/Mcpaeis/Liger/main/neumf/clusters.csv
Dataset lo

In [ ]:
small_test_data

,cell_id,sp,cell_1,OG0000000,OG0000001,OG0000004,OG0000007,OG0000008,OG0000010,OG0000011,OG0000013,OG0000014,OG0000016,OG0000017,OG0000018,OG0000019,OG0000020,OG0000021,OG0000022,OG0000023,OG0000024,OG0000026,OG0000027,OG0000028,OG0000030,OG0000031,OG0000032,OG0000033,OG0000034,OG0000035,OG0000038,OG0000039,OG0000041,OG0000043,OG0000044,OG0000045,OG0000046,OG0000049,OG0000050,OG0000051,...,OG0009194,OG0009195,OG0009196,OG0009197,OG0009198,OG0009199,OG0009200,OG0009201,OG0009202,OG0009203,OG0009204,OG0009205,OG0009206,OG0009207,OG0009208,OG0009209,OG0009210,OG0009212,OG0009214,OG0009215,OG0009216,OG0009217,OG0009218,OG0009219,OG0009220,OG0009221,OG0009224,OG0009225,OG0009226,OG0009227,OG0009228,OG0009229,OG0009230,OG0009231,OG0009232,OG0009233,OG0009234,OG0009235,OG0009236,OG0009237
0,Di_0_Di_C2_S27,Di,c2,1,4,8,0,11,5,0,100,7,1,0,27,83,0,1,4,4,0,14,8,45,0,4,0,7,0,5,1,0,0,1,2,9,308,1,4,2,...,0,0,0,0,0,0,0,4,1,1,0,0,5,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,5,0,0,0,1,0,0,4,0,0
1,Di_1_Di_C_C2,Di,c2,0,0,0,0,0,1,0,16,0,0,0,16,0,0,430,2,1,0,7,4,2,0,2,0,2,0,0,0,0,0,0,0,13,0,2,0,0,...,56,28,6,0,60,0,0,0,0,0,2,0,3,66,16,16,0,0,0,1,27,0,38,8,18,37,0,0,0,20,22,0,0,0,17,0,6,0,37,2
2,Di_10_Di_F_C22,Di,c2,126,0,25,1,159,115,8,393,10,6,0,277,99,0,895,340,44,15,162,213,51,0,131,0,1109,2,126,0,0,0,21,28,25,84,91,118,10,...,482,195,0,0,428,126,12,43,37,8,106,7,148,1074,283,133,6,30,0,57,398,34,1389,84,413,284,2,0,14,444,117,0,2,0,482,99,244,37,231,346
3,Di_11_Di_F_Si11,Di,Si1,14,9,1,0,8,0,0,133,0,0,0,9,19,0,69,23,12,0,5,187,0,0,483,0,99,0,8,0,0,0,0,0,2,2,2,0,9,...,56,35,0,0,22,5,0,0,1,0,1,0,219,56,16,2,0,0,0,0,11,0,49,8,12,10,4,0,0,9,11,0,0,0,8,0,25,0,37,9
4,Di_12_Di_F_Si12,Di,Si1,35,15,3,0,5,11,0,157,2,4,0,87,19,0,396,110,14,3,30,306,21,0,698,0,97,1,2,0,0,0,3,5,4,11,27,13,9,...,283,81,0,0,254,28,0,0,6,0,19,4,294,214,30,15,1,1,0,8,30,5,144,45,137,108,1,0,0,54,51,2,8,0,109,13,46,4,161,130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,Td_5_Td_3_Eye,Td,eye,1240,10019,3283,729,678,44,658,11203,210,34,6279,968,223,178,2576,129,278,162,53,195,2448,833,292,9,440,37,40,318,283,10,44,809,301,1003,24,42,38,...,55,10,5,14,6,63,11,23,8,19,230,13,3,58,85,3,14,5,9,17,38,16,274,18,183,35,18,0,4,82,35,8,19,22,99,53,67,20,58,5
161,Td_6_Td_4_C2,Td,c2,218,531,216,321,147,22,115,20,14,1,574,1225,72,45,13602,135,93,61,8,22,124,42,80,2,7,4,13,184,12,24,17,139,52,7,2,25,307,...,242,43,9,52,1003,150,13,354,108,68,308,18,0,1506,108,2,61,9,1,67,131,47,1183,113,486,66,76,1,3,438,794,27,42,6,155,96,978,10,172,19
162,Td_7_Td_4_Eye,Td,eye,1168,8681,2887,649,611,34,565,9520,174,23,5024,821,167,166,2395,69,235,167,34,170,1973,679,242,4,425,23,38,287,237,9,31,737,225,828,17,44,28,...,53,2,4,4,16,46,13,16,8,29,206,9,9,57,77,1,6,6,1,10,52,9,202,18,127,32,15,3,7,67,28,7,15,15,65,34,69,13,50,5
163,Td_8_Td_5_C2,Td,c2,1059,1475,615,143,847,0,291,140,930,0,853,848,51,2,13882,1209,417,141,0,8,323,460,45,0,1,5,3,78,8,0,33,5,76,62,1,0,341,...,103,7,0,317,37,522,0,30,677,527,9,347,0,374,1136,0,0,9,0,14,100,0,6256,66,197,23,1,184,0,49,326,18,0,0,48,0,886,0,89,35


In [ ]:
# remove the cells
X1 = liger.copy()
X2 = neumf.copy()
X3 = neunn.copy()
X4 = small_test_data.copy()
del X1['Unnamed: 0']
del X2['Unnamed: 0']
del X3['Unnamed: 0']
del X4['rowname']
X1 = X1.values.astype("float32")
X2 = X2.values.astype("float32")
X3 = X3.values.astype("float32")
X4 = X4.values.astype['float32']
# get the liger dimensions
l1, l2  = X1.shape
n1, n2  = X2.shape
nn1, nn2  = X3.shape
print("Number of liger rows: ", l1, " Number of liger features: ", l2)
print("Number of neumf neg rows: ", n1, " Number of neumf neg features: ", n2)
print("Number of neumf non-neg rows: ", nn1, " Number of neumf non-neg features: ", nn2)
# get the number of clusters
Y = (clusters.x).values
cls = len(np.unique(Y))
print("Number of clusters: ", cls)

Number of liger rows:  6332  Number of liger features:  20
Number of neumf neg rows:  6332  Number of neumf neg features:  20
Number of neumf non-neg rows:  6332  Number of neumf non-neg features:  20
Number of clusters:  14


In [ ]:
np.shape(origd)

(6332, 2814)

---
---
Build Model

---
---

In [ ]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
#len(dummy_y)
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, dummy_y, random_state = 11)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, dummy_y, random_state = 11)
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, dummy_y, random_state = 11)
X4_train, X4_test, y4_train, y4_test = train_test_split(origd, dummy_y, random_state = 11)

In [ ]:
def base_model(dim = 20):
  model = Sequential()
  model.add(Dense(1000, input_dim = dim, activation = "relu"))
  model.add(Dense(500, activation = "relu"))
  model.add(Dense(14, activation="softmax"))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # [tf.keras.metrics.AUC(multi_label = False)]
  return model

In [ ]:
n_epochs = 100
#estimator_l = KerasClassifier(build_fn=base_model, epochs=5, batch_size=5, verbose=1)
model_l = base_model()
#print(keras.utils.plot_model(model_l, "model_l.png", show_shapes=True))
model_l.fit(X1_train, y1_train, epochs = n_epochs, batch_size = 5, verbose=1)
#estimator_n = KerasClassifier(build_fn=base_model, epochs=5, batch_size=5, verbose=1)

Epoch 1/100
950/950 [==============================] - 19s 3ms/step - loss: 1.8594 - accuracy: 0.4070
Epoch 2/100
950/950 [==============================] - 3s 3ms/step - loss: 1.0299 - accuracy: 0.5975
Epoch 3/100
950/950 [==============================] - 3s 3ms/step - loss: 0.9186 - accuracy: 0.6295
Epoch 4/100
950/950 [==============================] - 3s 3ms/step - loss: 0.8764 - accuracy: 0.6291
Epoch 5/100
950/950 [==============================] - 3s 3ms/step - loss: 0.8410 - accuracy: 0.6263
Epoch 6/100
950/950 [==============================] - 3s 3ms/step - loss: 0.8331 - accuracy: 0.6331
Epoch 7/100
950/950 [==============================] - 3s 3ms/step - loss: 0.8300 - accuracy: 0.6369
Epoch 8/100
950/950 [==============================] - 3s 3ms/step - loss: 0.8270 - accuracy: 0.6356
Epoch 9/100
950/950 [==============================] - 3s 3ms/step - loss: 0.7963 - accuracy: 0.6447
Epoch 10/100
950/950 [==============================] - 3s 3ms/step - loss: 0.7913 - accur

In [ ]:
model_n = base_model()
model_n.fit(X2_train, y2_train, epochs = n_epochs, batch_size = 5, verbose=1)
#kfold = KFold(n_splits=10, shuffle=True)
#results = cross_val_score(estimator, X1, dummy_y, cv=kfold)
#print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/100
950/950 [==============================] - 4s 3ms/step - loss: 1.1029 - accuracy: 0.6060
Epoch 2/100
950/950 [==============================] - 3s 3ms/step - loss: 0.9241 - accuracy: 0.6324
Epoch 3/100
950/950 [==============================] - 3s 3ms/step - loss: 0.8937 - accuracy: 0.6570
Epoch 4/100
950/950 [==============================] - 3s 3ms/step - loss: 0.9016 - accuracy: 0.6498
Epoch 5/100
950/950 [==============================] - 3s 4ms/step - loss: 0.8821 - accuracy: 0.6702
Epoch 6/100
950/950 [==============================] - 3s 4ms/step - loss: 0.8741 - accuracy: 0.6704
Epoch 7/100
950/950 [==============================] - 3s 4ms/step - loss: 0.8588 - accuracy: 0.6749
Epoch 8/100
950/950 [==============================] - 3s 3ms/step - loss: 0.8356 - accuracy: 0.6876
Epoch 9/100
950/950 [==============================] - 3s 3ms/step - loss: 0.8419 - accuracy: 0.6897
Epoch 10/100
950/950 [==============================] - 3s 3ms/step - loss: 0.7938 - accura

In [ ]:
model_nn = base_model()
model_nn.fit(X3_train, y3_train, epochs = n_epochs, batch_size = 5, verbose=1)

Epoch 1/100
950/950 [==============================] - 4s 3ms/step - loss: 2.8173 - accuracy: 0.2743
Epoch 2/100
950/950 [==============================] - 3s 3ms/step - loss: 1.8789 - accuracy: 0.4196
Epoch 3/100
950/950 [==============================] - 3s 3ms/step - loss: 1.8745 - accuracy: 0.3950
Epoch 4/100
950/950 [==============================] - 3s 3ms/step - loss: 1.8710 - accuracy: 0.4008
Epoch 5/100
950/950 [==============================] - 3s 3ms/step - loss: 1.8371 - accuracy: 0.4125
Epoch 6/100
950/950 [==============================] - 3s 3ms/step - loss: 1.8435 - accuracy: 0.4000
Epoch 7/100
950/950 [==============================] - 3s 3ms/step - loss: 1.8240 - accuracy: 0.4065
Epoch 8/100
950/950 [==============================] - 3s 3ms/step - loss: 1.8435 - accuracy: 0.4048
Epoch 9/100
950/950 [==============================] - 3s 3ms/step - loss: 1.8378 - accuracy: 0.4000
Epoch 10/100
950/950 [==============================] - 3s 3ms/step - loss: 1.8294 - accura

In [ ]:
model_cb = base_model(dim=2814)
model_cb.fit(X4_train, y4_train, epochs = n_epochs, batch_size = 5, verbose=1)

Epoch 1/100
950/950 [==============================] - 5s 5ms/step - loss: 0.6997 - accuracy: 0.7807
Epoch 2/100
950/950 [==============================] - 4s 5ms/step - loss: 0.0879 - accuracy: 0.9718
Epoch 3/100
950/950 [==============================] - 4s 5ms/step - loss: 0.0423 - accuracy: 0.9862
Epoch 4/100
950/950 [==============================] - 4s 5ms/step - loss: 0.0182 - accuracy: 0.9957
Epoch 5/100
950/950 [==============================] - 4s 5ms/step - loss: 0.0252 - accuracy: 0.9931
Epoch 6/100
950/950 [==============================] - 4s 5ms/step - loss: 0.0141 - accuracy: 0.9953
Epoch 7/100
950/950 [==============================] - 4s 5ms/step - loss: 0.0053 - accuracy: 0.9983
Epoch 8/100
950/950 [==============================] - 4s 5ms/step - loss: 0.0018 - accuracy: 0.9999
Epoch 9/100
950/950 [==============================] - 4s 5ms/step - loss: 0.0758 - accuracy: 0.9851
Epoch 10/100
950/950 [==============================] - 4s 5ms/step - loss: 0.0104 - accura

In [ ]:
test_loss_l, test_acc_l = model_l.evaluate(X1_test,  y1_test, verbose=2)
test_loss_n, test_acc_n = model_n.evaluate(X2_test,  y2_test, verbose=2)
test_loss_nn, test_acc_nn = model_nn.evaluate(X3_test,  y3_test, verbose=2)
test_loss_cb, test_acc_cb = model_cb.evaluate(X4_test,  y4_test, verbose=2)

print('\nTest Liger accuracy:', test_acc_l) # based on the (H_liger, y) matrix
print('\nTest Neumf Neg accuracy:', test_acc_n) # based on the (H_nn, y) matrix but with the constraint removed
print('\nTest Neumf NonNeg accuracy:', test_acc_nn) # based on the (H_nn, y) matrix but with the constraint
print('\nTest Full Data accuracy:', test_acc_cb) # full datatset (X, y)

50/50 - 1s - loss: 1.0487 - accuracy: 0.7227
50/50 - 0s - loss: 0.6818 - accuracy: 0.7669
50/50 - 0s - loss: 1.8344 - accuracy: 0.4087
50/50 - 0s - loss: 1.5444 - accuracy: 0.9223

Test Liger accuracy: 0.7226784825325012

Test Neumf Neg accuracy: 0.7668982744216919

Test Neumf NonNeg accuracy: 0.40871763229370117

Test Full Data accuracy: 0.9222994446754456
